<h1>Normal Cross Validation</h1>

In [2]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('NFL Play by Play 2009-2018 (v5).csv')

C:\Users\aniga\AppData\Local\Temp\ipykernel_12256\1788553136.py:1: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NFL Play by Play 2009-2018 (v5).csv')


In [4]:
xy = df[[    'total_away_score',
    'posteam_score',
    'defteam_score',
    'score_differential',
    'yardline_100',
    'down',
    'goal_to_go',
    'ydstogo',
    'ydsnet',
    'quarter_seconds_remaining',
    'half_seconds_remaining',
    'game_seconds_remaining',
    'quarter_end',
    'play_type']]
xy = xy.dropna(subset = ['play_type'])
xy['down'] = xy['down'].fillna(0)

In [5]:
xy = df[[    'total_away_score',
    'posteam_score',
    'defteam_score',
    'score_differential',
    'yardline_100',
    'down',
    'goal_to_go',
    'ydstogo',
    'ydsnet',
    'quarter_seconds_remaining',
    'half_seconds_remaining',
    'game_seconds_remaining',
    'quarter_end',
        'play_type']]

In [6]:
xy = xy.dropna(subset = ['play_type'])

In [7]:
xy['down'] = xy['down'].fillna(0)

In [8]:
y = xy[['play_type']]
x = xy[['total_away_score',
    'posteam_score',
    'defteam_score',
    'score_differential',
    'yardline_100',
    'down',
    'goal_to_go',
    'ydstogo',
    'ydsnet',
    'quarter_seconds_remaining',
    'half_seconds_remaining',
    'game_seconds_remaining',
    'quarter_end']]

In [9]:
imputer = IterativeImputer()

In [10]:
x.isna().sum()

total_away_score                0
posteam_score                2545
defteam_score                2545
score_differential           2545
yardline_100                  368
down                            0
goal_to_go                      5
ydstogo                         0
ydsnet                          0
quarter_seconds_remaining       1
half_seconds_remaining         25
game_seconds_remaining         23
quarter_end                     0
dtype: int64

In [11]:
x.shape

(436497, 13)

In [12]:
x = imputer.fit_transform(x)

In [13]:
from sklearn.model_selection import cross_val_score

In [30]:
import torch
import torch.nn as nn
import numpy as np

In [31]:
database = torch.load("database.pt")

In [32]:
state = torch.tensor(database["state"], dtype = torch.float)
action = torch.tensor(database['action'].toarray(), dtype = torch.float)
reward = torch.tensor(database['reward'], dtype = torch.float)

In [14]:
model = RandomForestClassifier()

In [36]:
model.fit(state[:10000], action[:10000].argmax(-1))

RandomForestClassifier()

In [37]:
model.predict(x[:100])

array([6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 6, 3, 6, 2, 3, 2, 6, 2, 6, 6, 0, 0, 6, 0, 0, 0, 6, 0, 0, 6,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [38]:
dump(model, "RFMODEL.joblib")

['RFMODEL.joblib']

In [19]:
import pickle
from joblib import dump
from joblib import load

In [21]:
dump(model, "best_random_forest.joblib")

['best_random_forest.joblib']

In [14]:
scores = cross_val_score(model, x, y, cv=7)

C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was exp

In [15]:
for fold_idx, score in enumerate(scores):
    print(f"Fold {fold_idx+1} score: {score}")

Fold 1 score: 0.8528633513478839
Fold 2 score: 0.8558461760508043
Fold 3 score: 0.8576262488573857
Fold 4 score: 0.8543226903154417
Fold 5 score: 0.8532963420305659
Fold 6 score: 0.8533902110462506
Fold 7 score: 0.8368561164923984


<h1>Nested Cross Validation</h1>

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [23]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7]
}

In [13]:
model = RandomForestClassifier()

NameError: name 'RandomForestClassifier' is not defined

In [25]:
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_val, y_train_val)

C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was exp

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 150]})

In [26]:
best_rf_model = grid_search.best_estimator_

In [33]:
dump(best_rf_model, 'best_random_forest_model.joblib')

['best_random_forest_model.joblib']

In [35]:
nested_score = cross_val_score((best_rf_model), X_train_val, y_train_val, cv=5)

C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\aniga\miniconda3\envs\pytorch\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was exp

In [36]:
print(nested_score, nested_score.mean())

[0.79872566 0.79633448 0.79915234 0.79575882 0.7997394 ] 0.7979421381745515


In [10]:
from joblib import load

In [11]:
import pickle
from joblib import dump


In [12]:
loaded_rf_model = load('best_random_forest_model.joblib')

In [1]:
import torch

In [2]:
x = torch.load('database.pt')

In [7]:
x['state']

array([[0.        , 0.        , 0.        , ..., 0.4358232 , 0.87334788,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.4327664 , 0.87637661,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.42923466, 0.87974335,
        0.        ],
       ...,
       [0.09172607, 0.05350688, 0.09172607, ..., 0.48156188, 0.48156188,
        0.        ],
       [0.0969984 , 0.0565824 , 0.0969984 , ..., 0.46882561, 0.46882561,
        0.        ],
       [0.12761625, 0.07444281, 0.12761625, ..., 0.40411814, 0.40411814,
        0.        ]])

In [24]:
y = torch.load('database.pt')
y = y['action'].toarray()

In [16]:
model = RandomForestClassifier()

In [25]:
X_train_val, X_test, y_train_val, y_test = train_test_split(x['state'], y, test_size=0.2, random_state=42)

In [26]:
model.fit(X_train_val, y_train_val)
model.score(X_test, y_test)

0.6042895231090506